# Import libraries

In [1]:
import numpy as np
import cv2
import pickle

# Global variables

In [2]:
DATASET_FOLDER = "../theatre_dataset/"
VIDEO_FOLDER = "braunfels_1"
ASPECT_RATIO = 0.95 # 4/3 for carol_2, 0.95 for braunfels_1
OUTPUT_VIDEO_SIZE = (np.ceil(1280 / ASPECT_RATIO).astype(np.int16), 1280) # height, width

# Load files

In [3]:
file_video_input = cv2.VideoCapture(DATASET_FOLDER + VIDEO_FOLDER + "/" + VIDEO_FOLDER + ".mp4")
VIDEO_INPUT_CODEC = int(file_video_input.get(cv2.CAP_PROP_FOURCC)) # not working
VIDEO_INPUT_FRAME_SIZE = (int(file_video_input.get(cv2.CAP_PROP_FRAME_WIDTH)), int(file_video_input.get(cv2.CAP_PROP_FRAME_HEIGHT)))
VIDEO_INPUT_FPS = file_video_input.get(cv2.CAP_PROP_FPS)
VIDEO_INPUT_FRAMES_COUNT = int(file_video_input.get(cv2.CAP_PROP_FRAME_COUNT))
file_video_input.release()

In [4]:
VIDEO_INPUT_CODEC, VIDEO_INPUT_FRAME_SIZE, VIDEO_INPUT_FPS, VIDEO_INPUT_FRAMES_COUNT

(828601953, (3840, 2160), 29.97002997002997, 2278)

# Load the selected layouts and virtual camera positions

In [5]:
with open(DATASET_FOLDER + VIDEO_FOLDER + "/selected_layouts.pkl", "rb") as f:
    selected_layouts = pickle.load(f)
with open(DATASET_FOLDER + VIDEO_FOLDER + "/virtual_camera_positions.pkl", "rb") as f:
    virtual_camera_positions = pickle.load(f)

# Save the result

In [6]:
file_video_input = cv2.VideoCapture(DATASET_FOLDER + VIDEO_FOLDER + "/" + VIDEO_FOLDER + ".mp4")
file_video_output = cv2.VideoWriter(DATASET_FOLDER + VIDEO_FOLDER + "/" + 'output_ssc_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), VIDEO_INPUT_FPS, OUTPUT_VIDEO_SIZE[::-1])
for i in range(VIDEO_INPUT_FRAMES_COUNT):
    success, img_frame = file_video_input.read()
    img_frame_output = np.zeros(shape = OUTPUT_VIDEO_SIZE + (3, )).astype(np.uint8)
    # print(virtual_camera_positions[i], selected_layouts[i])

    if selected_layouts[i] == 0:
        coordinates = np.rint(virtual_camera_positions[i][0]).astype(np.int16)
        view_1 = img_frame[coordinates[1]-coordinates[2]:coordinates[1]+coordinates[2]+1, int(np.rint(coordinates[0]-ASPECT_RATIO*coordinates[2])):int(np.rint(coordinates[0]+ASPECT_RATIO*coordinates[2]))+1]
        view_2 = img_frame[coordinates[4]-coordinates[5]:coordinates[4]+coordinates[5]+1, int(np.rint(coordinates[3]-ASPECT_RATIO*coordinates[5])):int(np.rint(coordinates[3]+ASPECT_RATIO*coordinates[5]))+1]
        view_3 = img_frame[coordinates[7]-coordinates[8]:coordinates[7]+coordinates[8]+1, int(np.rint(coordinates[6]-ASPECT_RATIO*coordinates[8])):int(np.rint(coordinates[6]+ASPECT_RATIO*coordinates[8]))+1]

        view_1 = cv2.resize(view_1, (426, np.ceil(426 / ASPECT_RATIO).astype(np.int16)))
        view_2 = cv2.resize(view_2, (426, np.ceil(426 / ASPECT_RATIO).astype(np.int16)))
        view_3 = cv2.resize(view_3, (426, np.ceil(426 / ASPECT_RATIO).astype(np.int16)))

        row_start_no_1 = (OUTPUT_VIDEO_SIZE[0] - view_1.shape[0]) // 2
        row_start_no_2 = (OUTPUT_VIDEO_SIZE[0] - view_2.shape[0]) // 2
        row_start_no_3 = (OUTPUT_VIDEO_SIZE[0] - view_3.shape[0]) // 2
        img_frame_output[row_start_no_1:row_start_no_1+view_1.shape[0], :view_1.shape[1]] = view_1
        img_frame_output[row_start_no_2:row_start_no_2+view_2.shape[0], view_1.shape[1]+1:view_1.shape[1]+1+view_2.shape[1]] = view_2
        img_frame_output[row_start_no_3:row_start_no_3+view_3.shape[0], view_1.shape[1]+view_2.shape[1]+2:view_1.shape[1]+view_2.shape[1]+2+view_3.shape[1]] = view_3
    
    elif selected_layouts[i] == 1:
        coordinates = np.rint(virtual_camera_positions[i][0]).astype(np.int16)
        view_1 = img_frame[coordinates[1]-coordinates[2]:coordinates[1]+coordinates[2]+1, int(np.rint(coordinates[0]-ASPECT_RATIO*coordinates[2])):int(np.rint(coordinates[0]+ASPECT_RATIO*coordinates[2]))+1]
        coordinates = np.rint(virtual_camera_positions[i][1]).astype(np.int16)
        view_2 = img_frame[coordinates[1]-coordinates[2]:coordinates[1]+coordinates[2]+1, int(np.rint(coordinates[0]-ASPECT_RATIO*coordinates[2])):int(np.rint(coordinates[0]+ASPECT_RATIO*coordinates[2]))+1]

        view_1 = cv2.resize(view_1, (853, np.ceil(853 / ASPECT_RATIO).astype(np.int16)))
        view_2 = cv2.resize(view_2, (426, np.ceil(426 / ASPECT_RATIO).astype(np.int16)))

        row_start_no_1 = (OUTPUT_VIDEO_SIZE[0] - view_1.shape[0]) // 2
        row_start_no_2 = (OUTPUT_VIDEO_SIZE[0] - view_2.shape[0]) // 2
        img_frame_output[row_start_no_1:row_start_no_1+view_1.shape[0], :view_1.shape[1]] = view_1
        img_frame_output[row_start_no_2:row_start_no_2+view_2.shape[0], view_1.shape[1]+1:view_1.shape[1]+1+view_2.shape[1]] = view_2
    elif selected_layouts[i] == 2:
        coordinates = np.rint(virtual_camera_positions[i][1]).astype(np.int16)
        view_1 = img_frame[coordinates[1]-coordinates[2]:coordinates[1]+coordinates[2]+1, int(np.rint(coordinates[0]-ASPECT_RATIO*coordinates[2])):int(np.rint(coordinates[0]+ASPECT_RATIO*coordinates[2]))+1]
        coordinates = np.rint(virtual_camera_positions[i][0]).astype(np.int16)
        view_2 = img_frame[coordinates[1]-coordinates[2]:coordinates[1]+coordinates[2]+1, int(np.rint(coordinates[0]-ASPECT_RATIO*coordinates[2])):int(np.rint(coordinates[0]+ASPECT_RATIO*coordinates[2]))+1]

        view_1 = cv2.resize(view_1, (426, np.ceil(426 / ASPECT_RATIO).astype(np.int16)))
        view_2 = cv2.resize(view_2, (853, np.ceil(853 / ASPECT_RATIO).astype(np.int16)))

        row_start_no_1 = (OUTPUT_VIDEO_SIZE[0] - view_1.shape[0]) // 2
        row_start_no_2 = (OUTPUT_VIDEO_SIZE[0] - view_2.shape[0]) // 2
        img_frame_output[row_start_no_1:row_start_no_1+view_1.shape[0], :view_1.shape[1]] = view_1
        img_frame_output[row_start_no_2:row_start_no_2+view_2.shape[0], view_1.shape[1]+1:view_1.shape[1]+1+view_2.shape[1]] = view_2
    elif selected_layouts[i] == 3:
        coordinates = np.rint(virtual_camera_positions[i][0]).astype(np.int16)
        view_1 = img_frame[coordinates[1]-coordinates[2]:coordinates[1]+coordinates[2]+1, int(np.rint(coordinates[0]-ASPECT_RATIO*coordinates[2])):int(np.rint(coordinates[0]+ASPECT_RATIO*coordinates[2]))+1]

        view_1 = cv2.resize(view_1, (1280, np.ceil(1280 / ASPECT_RATIO).astype(np.int16)))

        row_start_no_1 = (OUTPUT_VIDEO_SIZE[0] - view_1.shape[0]) // 2
        img_frame_output[row_start_no_1:row_start_no_1+view_1.shape[0], :view_1.shape[1]] = view_1
    file_video_output.write(img_frame_output)
file_video_input.release()
file_video_output.release()